In [1]:
import os
import cv2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical 
# Note: TensorFlow/Keras imports are here as they'll be needed for the next step (EfficientNet)

In [12]:
def apply_augmentations(image):
    """
    Applies a random sequence of medical-safe augmentations (rotation, flip, brightness, zoom).
    """
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)

    # 1. Random Flip (Horizontal/Vertical)
    if np.random.rand() < 0.5:
        image = cv2.flip(image, 1)
    if np.random.rand() < 0.5:
        image = cv2.flip(image, 0)

    # 2. Small Random Rotation (±15 degrees)
    angle = np.random.uniform(-15, 15)
    M_rot = cv2.getRotationMatrix2D(center, angle, 1.0)
    image = cv2.warpAffine(image, M_rot, (w, h), borderMode=cv2.BORDER_CONSTANT)

    # 3. Random Brightness Adjustment (±20%)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    brightness_factor = np.random.uniform(0.8, 1.2)
    hsv[:, :, 2] = np.clip(hsv[:, :, 2] * brightness_factor, 0, 255).astype(np.uint8)
    image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    
    # 4. Small Random Zoom/Scale (0.9 to 1.1)
    scale = np.random.uniform(0.9, 1.1)
    M_scale = np.array([
        [scale, 0, 0], 
        [0, scale, 0]
    ], dtype=np.float32)
    M_scale[0, 2] = (1 - scale) * center[0]
    M_scale[1, 2] = (1 - scale) * center[1]
    image = cv2.warpAffine(image, M_scale, (w, h), borderMode=cv2.BORDER_CONSTANT)
    
    return image

# -------------------------------------------------------------------------------
# MODIFIED: load_and_map_labels function to ISOLATE TEST SAMPLES
# -------------------------------------------------------------------------------
def load_and_map_labels(csv_path, test_keyword='test'):
    """
    Loads the DR grading CSV and splits the data into train/val and test sets
    based on the presence of a keyword (e.g., 'test') in the image name.

    Returns:
        tuple: (df_train_val, df_test) DataFrames
    """
    try:
        df = pd.read_csv(csv_path, usecols=['id_code', 'diagnosis'])
        df.rename(columns={'id_code': 'Image_Name', 'diagnosis': 'DR_Grade'}, inplace=True)
        df['DR_Grade'] = df['DR_Grade'].astype(int) 
        
        print(f"✅ Successfully loaded {len(df)} total labels from CSV.")

        # Isolate Test Samples: Assuming images meant for testing have a keyword like 'test'
        # Since the IDRiD files usually don't contain 'test' keywords in the training CSV, 
        # this logic is set up to handle it if such files are present, or to use 
        # a standard random split if no keyword is found.
        
        # Check if the official IDRiD test set structure is in place (e.g., separate files)
        # Assuming for now your CSV is the only source.
        
        # If your 'test' images are identified by ID, you would update this condition
        df_test = df[df['Image_Name'].str.contains(test_keyword, case=False, na=False)]
        df_train_val = df[~df['Image_Name'].str.contains(test_keyword, case=False, na=False)]

        # Secondary split: If no 'test' images are found, reserve 10% randomly for testing
        if len(df_test) == 0:
            print("⚠️ No images with 'test' keyword found. Performing a 90/10 split for Train/Val vs. Test.")
            # Use stratify on the diagnosis to maintain class balance in the test set
            df_train_val, df_test = train_test_split(
                df, 
                test_size=0.1, 
                random_state=42, 
                stratify=df['DR_Grade']
            )
        
        print(f"   - Training/Validation Set Size: {len(df_train_val)}")
        print(f"   - Test Set Size: {len(df_test)}")
        print(f"Distribution of Grades (Train/Val):\n{df_train_val['DR_Grade'].value_counts().sort_index()}")
        
        return df_train_val, df_test

    except Exception as e:
        print(f"❌ Error during label loading and splitting: {e}")
        return None, None

# -------------------------------------------------------------------------------
# Modified: preprocess_and_augment function now handles both train/val and test logic
# -------------------------------------------------------------------------------

def preprocess_and_augment(df_labels, images_folder_path, target_size=(224, 224), target_max_samples=156, augment_flag=True):
    """
    Preprocesses, and optionally applies augmentation/oversampling.
    """
    image_data_list = []
    
    # Only calculate oversampling strategy if augmentation is enabled (for train/val set)
    if augment_flag:
        grade_counts = df_labels['DR_Grade'].value_counts().to_dict()
        needed_augmentations = {grade: max(0, target_max_samples - count) 
                                for grade, count in grade_counts.items()}
        print("\n--- Oversampling Strategy ---")
        print(f"Target Samples per Grade: {target_max_samples}")
        print(f"Needed augmentations (per grade total): {needed_augmentations}")
    else:
        # For the test set, no oversampling needed
        needed_augmentations = {grade: 0 for grade in df_labels['DR_Grade'].unique()}


    total_augmentations_applied = 0
    
    for grade in sorted(df_labels['DR_Grade'].unique()):
        grade_df = df_labels[df_labels['DR_Grade'] == grade]
        current_count = len(grade_df)
        
        augmentation_multiplier = 0
        if augment_flag and current_count < target_max_samples and current_count > 0:
            augmentation_multiplier = int(np.ceil(needed_augmentations[grade] / current_count))
        
        for index, row in grade_df.iterrows():
            img_name_id = row['Image_Name']
            dr_grade = row['DR_Grade']
            img_path = os.path.join(images_folder_path, img_name_id + '.jpg')
            
            img = cv2.imread(img_path)
            
            if img is None:
                # Still handle the image read errors
                # print(f"⚠️ Warning: Could not read image {img_path}. Skipping.")
                continue

            img_resized = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)
            
            # 1. ORIGINAL image
            # Store the data with a unique filename identifier for the final saving step
            image_data_list.append({
                'image': img_resized / 255.0, # Normalized image array
                'label': dr_grade,           # Integer label
                'file_id': f"{img_name_id}_grade_{dr_grade}_original"
            })
            
            # 2. Generate AUGMENTED copies (only if augment_flag is True)
            if augment_flag:
                for i in range(augmentation_multiplier):
                    augmented_img = apply_augmentations(img_resized.copy()) 
                    
                    image_data_list.append({
                        'image': augmented_img / 255.0,
                        'label': dr_grade,
                        'file_id': f"{img_name_id}_grade_{dr_grade}_aug{i+1}"
                    })
                    total_augmentations_applied += 1
    
    # 3. Final Array Conversion
    X = np.array([d['image'] for d in image_data_list], dtype='float32')
    y_int = np.array([d['label'] for d in image_data_list], dtype='int32')
    y_onehot = to_categorical(y_int, num_classes=5)
    file_ids = np.array([d['file_id'] for d in image_data_list])
    
    print(f"\n--- Preprocessing Summary ({'Augmented' if augment_flag else 'Test'}) ---")
    print(f"Total processed samples: {len(X)}")
    if augment_flag:
        print(f"Total augmentations created: {total_augmentations_applied}")
    print(f"Final Distribution of Grades:\n{pd.Series(y_int).value_counts().sort_index()}")
    
    return X, y_onehot, file_ids, y_int

# -------------------------------------------------------------------------------
# save_split_images function remains unchanged (saves images from arrays to disk)
# -------------------------------------------------------------------------------
def save_split_images(X_set, y_int_set, file_id_set, output_base_path, folder_name):
    """Saves the image arrays to the specified train/validation/test folder."""
    target_folder = os.path.join(output_base_path, folder_name)
    os.makedirs(target_folder, exist_ok=True)
    
    print(f"\nSaving {len(X_set)} images to: {target_folder}")
    
    for i, (img_array, grade, file_id) in enumerate(zip(X_set, y_int_set, file_id_set)):
        img_denorm = (img_array * 255.0).astype(np.uint8)
        save_path = os.path.join(target_folder, f"{file_id}.png")
        cv2.imwrite(save_path, img_denorm)
        
    print(f"Successfully saved {len(X_set)} files to {folder_name} folder.")

In [14]:
if __name__ == "__main__":
    # 📌 IMPORTANT: Replace the 'path/to...' parts with your actual local file system structure
    # Based on your screenshot:
    BASE_PATH = r'datasets\IDRiD_diabities' # The root folder you showed
    
    CSV_FILE_PATH = os.path.join(BASE_PATH, 'idrid_labels.csv') 
    IMAGES_FOLDER_PATH = r'datasets\IDRiD_diabities\Imagenes\Imagenes' # Corrected path for images
    OUTPUT_BASE_PATH = r'datasets\Preprocessed_images_output'

    
# 1. Load Labels and Split into Train/Val and Test
    df_train_val, df_test = load_and_map_labels(CSV_FILE_PATH)

    if df_train_val is not None:
        
        # 2. Process and Augment the Train/Validation Set
        X_tv, y_tv, file_ids_tv, y_int_tv = preprocess_and_augment(
            df_labels=df_train_val,
            images_folder_path=IMAGES_FOLDER_PATH,
            target_size=(224, 224), 
            target_max_samples=125,
            augment_flag=True # Enable augmentation/oversampling
        )
        
        # 3. Split the Augmented Data into Training and Validation Arrays (e.g., 80/20)
        X_train, X_val, y_train, y_val, file_ids_train, file_ids_val, y_int_train, y_int_val = train_test_split(
            X_tv, y_tv, file_ids_tv, y_int_tv, 
            test_size=0.2, 
            random_state=42, 
            stratify=y_int_tv # Stratify using the integer labels of the full balanced set
        )
        
        # 4. Process the Test Set (No Augmentation)
        X_test, y_test, file_ids_test, y_int_test = preprocess_and_augment(
            df_labels=df_test,
            images_folder_path=IMAGES_FOLDER_PATH,
            target_size=(224, 224), 
            target_max_samples=0, # Ignored, but explicit
            augment_flag=False # CRUCIAL: Disable augmentation
        )

        # 5. Save the split images to disk
        print("\n--- Saving Split Images to Disk ---")
        
        # Save Training and Validation sets
        save_split_images(X_train, y_int_train, file_ids_train, OUTPUT_BASE_PATH, 'train')
        save_split_images(X_val, y_int_val, file_ids_val, OUTPUT_BASE_PATH, 'validation')
        
        # Save Test set
        save_split_images(X_test, y_int_test, file_ids_test, OUTPUT_BASE_PATH, 'test')


        print("\n--- ✅ Final Data Preparation Complete ---")
        print(f"Training Set Size: {len(X_train)}")
        print(f"Validation Set Size: {len(X_val)}")
        print(f"Test Set Size: {len(X_test)}")
        print(f"Images are now saved in: {OUTPUT_BASE_PATH}/train, /validation, and /test")

✅ Successfully loaded 455 total labels from CSV.
   - Training/Validation Set Size: 375
   - Test Set Size: 80
Distribution of Grades (Train/Val):
DR_Grade
0    114
1     17
2    125
3     64
4     55
Name: count, dtype: int64

--- Oversampling Strategy ---
Target Samples per Grade: 125
Needed augmentations (per grade total): {2: 0, 0: 11, 3: 61, 4: 70, 1: 108}

--- Preprocessing Summary (Augmented) ---
Total processed samples: 782
Total augmentations created: 407
Final Distribution of Grades:
0    228
1    136
2    125
3    128
4    165
Name: count, dtype: int64

--- Preprocessing Summary (Test) ---
Total processed samples: 80
Final Distribution of Grades:
0    15
1     5
2    31
3    20
4     9
Name: count, dtype: int64

--- Saving Split Images to Disk ---

Saving 625 images to: datasets\Preprocessed_images_output\train
Successfully saved 625 files to train folder.

Saving 157 images to: datasets\Preprocessed_images_output\validation
Successfully saved 157 files to validation folder.

In [7]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# --- 1. GPU/TF Environment Setup ---
print("TensorFlow device in use:", tf.config.list_physical_devices())
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Allow memory growth for efficient GPU memory usage
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ GPU memory growth set.")
    except Exception as e:
        print(f"⚠️ Could not set GPU memory growth: {e}")

# --- 2. Data Generator Class ---

class ImageBatchGenerator(Sequence):
    """
    Keras Sequence to generate batches of images and labels from disk, 
    efficiently handling large datasets like the augmented IDRiD set.
    """
    def __init__(self, folder_path, batch_size=16, num_classes=5, shuffle=True):
        # List all PNG files in the directory
        self.file_list = [f for f in os.listdir(folder_path) if f.endswith('.png')]
        self.folder_path = folder_path
        self.batch_size = batch_size
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        
        if not self.file_list:
             raise FileNotFoundError(f"❌ No .png files found in the folder: {folder_path}")

    def __len__(self):
        # Number of batches per epoch
        return int(np.ceil(len(self.file_list) / self.batch_size))
        
    def __getitem__(self, idx):
        # Generate indices for the batch
        batch_files = self.file_list[idx * self.batch_size:(idx + 1) * self.batch_size]
        images, labels = [], []
        
        for fname in batch_files:
            # Load image and convert from BGR (cv2 default) to RGB
            img = cv2.imread(os.path.join(self.folder_path, fname))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            # Normalize to [0, 1]
            images.append(img / 255.0)
            
            # Extract grade from filename: ..._grade_{label}_...
            try:
                # Assuming filename format is 'ImageName_grade_X_suffix.png'
                grade_str = fname.split('_grade_')[1].split('_')[0]
                grade = int(grade_str)
            except IndexError:
                # Handle cases where the filename format might be unexpected
                print(f"⚠️ Could not parse grade from filename: {fname}. Assuming grade 0.")
                grade = 0
                
            labels.append(grade)
            
        images = np.array(images, dtype='float32')
        # One-hot encode the labels
        labels = tf.keras.utils.to_categorical(labels, num_classes=self.num_classes)
        return images, labels
        
    def on_epoch_end(self):
        # Shuffle file list after each epoch
        if self.shuffle:
            np.random.shuffle(self.file_list)

# --- 3. Model Building Function ---

def build_efficientnet_model(input_shape, num_classes):
    """Builds the EfficientNetB0 model with a custom classification head."""
    input_tensor = Input(shape=input_shape)
    
    # 1. Load pre-trained base model
    base_model = EfficientNetB0(
        include_top=False, 
        weights='imagenet', 
        input_tensor=input_tensor
    )
    # Freeze the base model for Phase 1
    base_model.trainable = False 
    
    # 2. Add custom classification head
    x = GlobalAveragePooling2D(name="avg_pool")(base_model.output)
    x = Dropout(0.5)(x) 
    output = Dense(num_classes, activation='softmax', name='predictions')(x)
    
    model = Model(inputs=base_model.input, outputs=output)
    return model

# --- 4. Main Execution Block ---

# --- DATA PATHS ---
# NOTE: Ensure these paths match where you saved your preprocessed data
# Example path structure assumed: BASE_PATH/Preprocessed_images_output/{train,validation,test}
OUTPUT_BASE_PATH = r'datasets\Preprocessed_images_output' 

train_folder = os.path.join(OUTPUT_BASE_PATH, 'train')
val_folder = os.path.join(OUTPUT_BASE_PATH, 'validation')
test_folder = os.path.join(OUTPUT_BASE_PATH, 'test')

batch_size = 16
INPUT_SHAPE = (224, 224, 3)
NUM_CLASSES = 5

# Create Generators
train_gen = ImageBatchGenerator(train_folder, batch_size=batch_size, shuffle=True)
val_gen = ImageBatchGenerator(val_folder, batch_size=batch_size, shuffle=False)
test_gen = ImageBatchGenerator(test_folder, batch_size=batch_size, shuffle=False)

# Build Model
model = build_efficientnet_model(INPUT_SHAPE, NUM_CLASSES)

# --- Callbacks ---
# Saving ONLY weights to avoid the JSON serialization/EagerTensor error
checkpoint_path = 'efficientnet_idrid_best_weights_only.h5'
checkpoint = ModelCheckpoint(
    checkpoint_path, 
    monitor='val_loss', 
    verbose=1, 
    save_best_only=True, 
    save_weights_only=True, 
    mode='min'
)
# Patience increased to 8 for more resilient training
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=8, 
    verbose=1, 
    mode='min', 
    restore_best_weights=True
)
callbacks_list = [checkpoint, early_stopping]

# --------------------------------------------------------------------------------
# PHASE 1: Train Classification Head (Frozen Base)
# --------------------------------------------------------------------------------
print("\n--- Phase 1: Training Classification Head (Frozen Base) ---")

model.compile(
    optimizer=Adam(learning_rate=1e-3), # Higher LR (1e-3) for new layers
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20, # Initial training epochs increased
    callbacks=callbacks_list
)

# --------------------------------------------------------------------------------
# PHASE 2: Fine-Tuning (Unfreeze and Retrain)
# --------------------------------------------------------------------------------
print("\n--- Phase 2: Fine-Tuning (Unfreeze and Retrain) ---")

# --- CRITICAL FIX: Create new generators with a smaller batch size (e.g., 8) ---
fine_tune_batch_size = 8 
print(f"Reduced batch size to {fine_tune_batch_size} for fine-tuning to prevent OOM errors.")

train_gen_ft = ImageBatchGenerator(train_folder, batch_size=fine_tune_batch_size, shuffle=True)
val_gen_ft = ImageBatchGenerator(val_folder, batch_size=fine_tune_batch_size, shuffle=False)

# Unfreeze the base model
model.trainable = True

# Load the best weights from Phase 1 before starting fine-tuning
# ... (loading weights code) ...
    
# Re-compile with a much lower learning rate
model.compile(
    optimizer=Adam(learning_rate=1e-5), # Very low LR (1e-5) for fine-tuning
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Use the new, smaller generators for the fit call
history_finetune = model.fit(
    train_gen_ft,             # <-- Use the smaller batch size generator
    validation_data=val_gen_ft, # <-- Use the smaller batch size generator
    epochs=50, 
    initial_epoch=history.epoch[-1] + 1 if history.epoch else 0,
    callbacks=callbacks_list
)

# --------------------------------------------------------------------------------
# 5. FINAL EVALUATION AND SAVING
# --------------------------------------------------------------------------------
print("\n--- Final Evaluation on Test Set ---")
# Load the overall best weights saved during both phases
model.load_weights(checkpoint_path) 

test_loss, test_acc = model.evaluate(test_gen, verbose=1)
print(f"\nFinal Test Accuracy: {test_acc:.4f}")

# Save the final model in the SavedModel format (robust against serialization errors)
SAVE_PATH = 'efficientnet_idrid_final_savedmodel'
model.save(SAVE_PATH)
print(f"🎉 Model saved to {SAVE_PATH} folder (SavedModel format).")

TensorFlow device in use: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
✅ GPU memory growth set.

--- Phase 1: Training Classification Head (Frozen Base) ---
Epoch 1/20
40/40 [==============================] - ETA: 0s - loss: 1.6384 - accuracy: 0.2656
Epoch 1: val_loss improved from inf to 1.59669, saving model to efficientnet_idrid_best_weights_only.h5
40/40 [==============================] - 8s 88ms/step - loss: 1.6384 - accuracy: 0.2656 - val_loss: 1.5967 - val_accuracy: 0.2102
Epoch 2/20
39/40 [============================>.] - ETA: 0s - loss: 1.6329 - accuracy: 0.2397
Epoch 2: val_loss did not improve from 1.59669
40/40 [==============================] - 2s 50ms/step - loss: 1.6270 - accuracy: 0.2416 - val_loss: 1.6057 - val_accuracy: 0.2102
Epoch 3/20
39/40 [============================>.] - ETA: 0s - loss: 1.6404 - accuracy: 0.2365
Epoch 3: val_loss did not improve from 1.59669
40/40 [========

INFO:tensorflow:Assets written to: efficientnet_idrid_final_savedmodel\assets


INFO:tensorflow:Assets written to: efficientnet_idrid_final_savedmodel\assets


TypeError: Unable to serialize [2.0896919 2.1128857 2.1081853] to JSON. Unrecognized type <class 'tensorflow.python.framework.ops.EagerTensor'>.